Code to Calculate Prices: European Options (Monte-Carlo Methods)

In [1]:
import numpy as np

# Parameters
S0 = 100  # Initial stock price
K = 100   # Strike price
T = 0.25  # Time to maturity (3 months)
r = 0.05  # Risk-free rate
sigma = 0.20  # Volatility
n_simulations = 100000  # Number of Monte-Carlo simulations
n_steps = int(T * 252)  # Daily time steps

# Time increment
dt = T / n_steps

# Monte-Carlo simulation
np.random.seed(42)
Z = np.random.standard_normal((n_simulations, n_steps))
S = np.zeros((n_simulations, n_steps + 1))
S[:, 0] = S0
for t in range(1, n_steps + 1):
    S[:, t] = S[:, t-1] * np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z[:, t-1])

# Payoffs
call_payoff = np.maximum(S[:, -1] - K, 0)
put_payoff = np.maximum(K - S[:, -1], 0)

# Discounted prices
call_price = np.exp(-r * T) * np.mean(call_payoff)
put_price = np.exp(-r * T) * np.mean(put_payoff)

call_price, put_price


(4.584767794205709, 3.3647051996996353)

Q6: Calculate Delta for European Options
Delta is calculated by shocking the underlying asset price.

In [2]:
epsilon = 1e-4  # Small change in S0

# Shock the initial price
S0_up = S0 + epsilon

# Re-run the simulation for the shocked price
S_up = np.zeros((n_simulations, n_steps + 1))
S_up[:, 0] = S0_up
for t in range(1, n_steps + 1):
    S_up[:, t] = S_up[:, t-1] * np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z[:, t-1])

# Payoffs with shocked price
call_payoff_up = np.maximum(S_up[:, -1] - K, 0)
put_payoff_up = np.maximum(K - S_up[:, -1], 0)

# Calculate Delta
delta_call = (np.exp(-r * T) * np.mean(call_payoff_up) - call_price) / epsilon
delta_put = (np.exp(-r * T) * np.mean(put_payoff_up) - put_price) / epsilon

delta_call, delta_put


(0.5692625808517704, -0.4305158456263669)

Q7: Calculate Vega for European Options
Vega is calculated by shocking the volatility.

In [3]:
sigma_up = sigma + epsilon

# Simulate paths with increased volatility
paths_vega = np.zeros((n_simulations, n_steps + 1))
paths_vega[:, 0] = S0

for t in range(1, n_steps + 1):
    z = np.random.standard_normal(n_simulations)
    paths_vega[:, t] = paths_vega[:, t-1] * np.exp((r - 0.5 * sigma_up**2) * dt + sigma_up * np.sqrt(dt) * z)

# Calculate payoffs
call_payoff_vega = np.maximum(paths_vega[:, -1] - K, 0)
put_payoff_vega = np.maximum(K - paths_vega[:, -1], 0)

# Calculate Vega
vega_call = (np.mean(call_payoff_vega) - np.mean(call_payoff)) / epsilon
vega_put = (np.mean(put_payoff_vega) - np.mean(put_payoff)) / epsilon

print(f"Vega for European Call Option: {vega_call:.4f}")
print(f"Vega for European Put Option: {vega_put:.4f}")


Vega for European Call Option: 259.1190
Vega for European Put Option: 154.3636


Step 2: American Options (Monte-Carlo Methods)
Q5: Price the American Put Option
For American options, you need to account for the possibility of early exercise at each time step.

In [5]:
# Function to calculate American Put option price
def american_put_price(S0, K, r, sigma, T, n_steps, n_simulations):
    dt = T / n_steps
    Z = np.random.standard_normal((n_simulations, n_steps))
    S = np.zeros((n_simulations, n_steps + 1))
    S[:, 0] = S0
    payoff = np.zeros((n_simulations, n_steps + 1))
    
    for t in range(1, n_steps + 1):
        S[:, t] = S[:, t-1] * np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z[:, t-1])
        payoff[:, t] = np.maximum(K - S[:, t], 0)  # Early exercise value
    
    payoff[:, -1] = np.maximum(K - S[:, -1], 0)  # Payoff at maturity
    discounted_payoff = np.exp(-r * T) * np.max(payoff, axis=1)
    
    return np.mean(discounted_payoff)

american_put_price(S0, K, r, sigma, T, n_steps, n_simulations)


6.421956755971517

Q6: Calculate Delta for the American Put Option
Delta is calculated similarly as before by shocking the underlying asset price.

In [6]:
delta_am_put = (american_put_price(S0 + epsilon, K, r, sigma, T, n_steps, n_simulations) - american_put_price(S0, K, r, sigma, T, n_steps, n_simulations)) / epsilon

delta_am_put


356.3220516830157

Q7: Calculate Vega for the American Put Option
Vega is calculated similarly as before by shocking the volatility.

In [7]:
vega_am_put = (american_put_price(S0, K, r, sigma + epsilon, T, n_steps, n_simulations) - american_put_price(S0, K, r, sigma, T, n_steps, n_simulations)) / epsilon

vega_am_put


140.75759672379283

Step 3: Exotic Option Pricing (Monte-Carlo Methods)
Price an Up-and-Out (UAO) Barrier Option

In [8]:
# Parameters for UAO barrier option
S0 = 120  # Initial stock price
K = 120   # Strike price (ATM)
T = 8/12  # Time to maturity (8 months)
r = 0.06  # Risk-free rate
sigma = 0.30  # Volatility
barrier = 141  # Barrier level

# Monte-Carlo simulation for UAO
def uao_barrier_option_price(S0, K, r, sigma, T, barrier, n_simulations, n_steps):
    dt = T / n_steps
    Z = np.random.standard_normal((n_simulations, n_steps))
    S = np.zeros((n_simulations, n_steps + 1))
    S[:, 0] = S0
    
    for t in range(1, n_steps + 1):
        S[:, t] = S[:, t-1] * np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z[:, t-1])
    
    payoff = np.where(np.max(S, axis=1) < barrier, np.maximum(S[:, -1] - K, 0), 0)
    return np.exp(-r * T) * np.mean(payoff)

uao_price = uao_barrier_option_price(S0, K, r, sigma, T, barrier, n_simulations, n_steps)
uao_price


0.7955716279309807